In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import string
  
 

data = pd.read_csv("../../Data/SMSSpamCollection.txt", sep='\t', header=None)
data.columns = ['label', 'Content']

en_stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data['Content_len'] = data['Content'].apply(lambda x: len(x) - x.count(" "))

def count_punctuation(text):
    binary_array = [1 for ch in text if ch in string.punctuation] 
    nb_ponctuation = sum(binary_array)
    total = len(text) - text.count(" ")
    return round(nb_ponctuation/(total), 4)*100

data['punctuation_rate'] = data['Content'].apply(lambda x: count_punctuation(x))


def clean_email(email):
    result = "".join([word for word in email if word not in string.punctuation])
    tokens = re.split('\W+', result)
    text = [ps.stem(word) for word in tokens if word not in en_stopwords]
    return text


vectorisation_full = CountVectorizer(analyzer=clean_email)
vect_final = vectorisation_full.fit_transform(data['Content'])

all_data = pd.concat([pd.DataFrame(vect_final.toarray()), data['Content_len'], data['punctuation_rate']], axis=1)

In [2]:
all_data

,0,1,2,3,4,5,6,7,8,9,...,8332,8333,8334,8335,8336,8337,8338,8339,Content_len,punctuation_rate
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,92,9.78
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,24,25.00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,128,4.69
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,39,15.38
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,131,6.11
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,29,3.45
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,48,14.58
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,100,1.00


In [3]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(all_data, data['label'], test_size=0.2)

In [5]:
from sklearn  import svm

In [6]:
alg_svm= svm.SVC(kernel = 'linear')

In [40]:
alg_svm.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [41]:
predictions = alg_svm.predict(X_test)

In [42]:
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [43]:
len(predictions)

1115

In [44]:
len(Y_test)

1115

In [50]:
len(predictions[predictions=='spam'])

121

In [51]:
len(Y_test[Y_test=='spam'])

125